In [1]:
import numpy as np
import random
import os
import time
import torch 
from torch.autograd import Variable
from copy import deepcopy
from random import shuffle

In [2]:
from lib.turn_program_into_file import turn_program_into_file
from game.wrappers.game_container import GameContainer

In [3]:
cpus = 4
player_size = 8
game = GameContainer(128, 128)
# Each run taks 2.6 sec per cpu, 38089 = a 24h run.
run_limit = 10
save_cycle = 2
output_dir = "./output/pyramid/"
run = int(time.time())
save_dir = output_dir + str(run) + '/'

In [4]:
# king_hill_ids = list(range(0, cpus))
os.makedirs(os.path.join(output_dir, str(run)))

In [ ]:
# -*- coding: utf-8 -*-
import torch

top = 2

# Use the nn package to define our model and loss function.
p1 = torch.nn.Sequential(
    torch.nn.Linear(top**0, top**1),
    torch.nn.ReLU(),
    torch.nn.Linear(top**1, top**2),
    torch.nn.ReLU(),
    torch.nn.Linear(top**2, top**3),
    torch.nn.ReLU(),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.4),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.4),
    torch.nn.Linear(top**5, top**6),
    torch.nn.Sigmoid(),
)


p2 = torch.nn.Sequential(
    torch.nn.Linear(top**0, top**1),
    torch.nn.ReLU(),
    torch.nn.Linear(top**1, top**2),
    torch.nn.ReLU(),
    torch.nn.Linear(top**2, top**3),
    torch.nn.ReLU(),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(top**5, top**6),
    torch.nn.Sigmoid(),
)


loss_fn = torch.nn.BCELoss()


def return_result(game_result, number):
    result = 0
    
    for i in game_result:
        if i == number:
            result += 1
        
    return result

def create_player(game_board_pt, player_size):
    game_board_np = game_board_pt.detach().numpy()
    
    for i in range(len(game_board_np)):
        game_board_np[i] = 1 if game_board_np[i] > 0.5 else 0
    
    game_board_np = np.array(game_board_np, dtype=np.int)
    
    return game_board_np


def run_game(home, away, rounds, player_size):
    home = np.reshape(home, (player_size, player_size))
    away = np.reshape(away, (player_size, player_size))
    
    game.add_players(home, away)
    game.launch(rounds)
    final_board = game.gol.grid()
    final_board = np.array(final_board).flatten()
    
    return return_result(final_board, 1)

def create_loss_board(home, away, rounds, player_size):
    org_score = run_game(home, away, rounds, player_size)
    loss_board = deepcopy(home)
    tile_board = deepcopy(home)
    top_score = org_score
    check_order = list(range(0, player_size**2))
    shuffle(check_order)
    
    for i in check_order:
        tile_board[i] = 0 if home[i] == 1 else 1
        tile_result = run_game(tile_board, away, rounds, player_size)
        if tile_result >= top_score:
            loss_board[i] = tile_board[i]
            top_score = tile_result
        else:
            tile_board = deepcopy(loss_board)
    
    return torch.tensor(loss_board, dtype=torch.float), org_score, top_score

learning_rate = 0.0001
optimizer_p1 = torch.optim.Adam(p1.parameters(), lr=learning_rate)
optimizer_p2 = torch.optim.Adam(p2.parameters(), lr=learning_rate)

for t in range(10000):
    
    
    start_p1 = Variable(torch.tensor([1.0], requires_grad=True))
    start_p2 = Variable(torch.tensor([1.0], requires_grad=True))
    
    p1_board = p1(start_p1)
    p2_board = p2(start_p2)

    with torch.no_grad():
        p1_board_np = create_player(p1_board.clone(), player_size)
        p2_board_np = create_player(p2_board.clone(), player_size)

        p1_loss_board, p1_score, p1_top = create_loss_board(p1_board_np, p2_board_np, 100, player_size)
        p2_loss_board, p2_score, p2_top = create_loss_board(p2_board_np, p1_board_np, 100, player_size)

    loss_p1 = loss_fn(p1_board, p1_loss_board)
    loss_p1.backward()

    
    optimizer_p1.step()
    
    loss_p2 = loss_fn(p2_board, p2_loss_board)
    loss_p2.backward()
    optimizer_p2.step()
    
    optimizer_p1.zero_grad()
    optimizer_p2.zero_grad()
    
    print(loss_p1, p1_score, p1_top)
    print(loss_p2, p2_score, p2_top)
    
    if t % 10 == 0:
        print("Saving players:")
        print(p1_board, p2_board)
        turn_program_into_file(np.reshape(p1_board_np, (8,8)), save_dir + "p1-cycle-" + str(t) + ".rle", "RKH", "EW", "")
        turn_program_into_file(np.reshape(p2_board_np, (8,8)), save_dir + "p2-cycle-" + str(t) + ".rle", "RKH", "EW", "")

tensor(0.6449) 122 130
tensor(0.6440) 82 140
Saving players:
tensor([ 0.4417,  0.5753,  0.4806,  0.4976,  0.5055,  0.5274,  0.4717,
         0.4844,  0.4976,  0.4843,  0.4717,  0.5314,  0.4439,  0.5377,
         0.4534,  0.4468,  0.4635,  0.4688,  0.5405,  0.5225,  0.5054,
         0.5429,  0.5160,  0.4806,  0.5277,  0.4764,  0.5433,  0.4525,
         0.4647,  0.4776,  0.4691,  0.5264,  0.4570,  0.5203,  0.5073,
         0.4591,  0.4642,  0.5453,  0.5208,  0.5162,  0.4612,  0.4595,
         0.5212,  0.5245,  0.5805,  0.5459,  0.4755,  0.5193,  0.5042,
         0.4951,  0.4950,  0.5000,  0.4893,  0.4845,  0.5512,  0.4593,
         0.5317,  0.5017,  0.4752,  0.4470,  0.5209,  0.5111,  0.4511,
         0.4351]) tensor([ 0.5258,  0.5403,  0.4868,  0.4988,  0.5398,  0.4948,  0.4820,
         0.4721,  0.5200,  0.5362,  0.4144,  0.4356,  0.5138,  0.5110,
         0.4564,  0.4869,  0.4569,  0.4887,  0.4877,  0.5024,  0.4807,
         0.5003,  0.5169,  0.4947,  0.4876,  0.4795,  0.4490,  0.4398

tensor(0.6474) 6 141
tensor(0.6545) 12 222
tensor(0.6563) 38 111
tensor(0.6419) 30 150
tensor(0.6545) 65 123
tensor(0.6512) 58 204
tensor(0.6314) 94 137
tensor(0.6447) 77 158
tensor(0.6659) 5 167
tensor(0.6458) 24 172
tensor(0.6420) 83 130
tensor(0.6451) 19 178
tensor(0.6528) 36 215
tensor(0.6619) 12 253
tensor(0.6515) 12 126
tensor(0.6431) 59 155
tensor(0.6527) 30 168
tensor(0.6438) 4 157
tensor(0.6608) 8 224
tensor(0.6506) 20 180
Saving players:
tensor([ 0.4496,  0.5711,  0.4828,  0.4998,  0.4853,  0.5145,  0.4767,
         0.5163,  0.5245,  0.4656,  0.5138,  0.4974,  0.4461,  0.5846,
         0.4698,  0.4356,  0.4494,  0.5052,  0.5410,  0.5385,  0.5414,
         0.5342,  0.5193,  0.4559,  0.4903,  0.5079,  0.5170,  0.4585,
         0.4389,  0.4915,  0.4580,  0.5560,  0.4671,  0.5203,  0.5462,
         0.4506,  0.4541,  0.5448,  0.4761,  0.4956,  0.5190,  0.4429,
         0.4709,  0.4949,  0.5974,  0.5504,  0.5207,  0.5406,  0.5286,
         0.4680,  0.5060,  0.4816,  0.5253,  0.4831

tensor(0.6335) 58 143
tensor(0.6344) 11 205
tensor(0.6436) 73 139
tensor(0.6394) 72 110
tensor(0.6382) 49 179
tensor(0.6333) 41 177
tensor(0.6428) 0 122
tensor(0.6484) 4 169
tensor(0.6631) 131 179
tensor(0.6483) 18 165
tensor(0.6640) 69 156
tensor(0.6312) 64 183
tensor(0.6453) 7 177
tensor(0.6403) 31 122
tensor(0.6472) 4 166
tensor(0.6369) 99 178
tensor(0.6402) 77 145
tensor(0.6353) 12 148
tensor(0.6662) 69 179
tensor(0.6440) 73 175
Saving players:
tensor([ 0.4676,  0.5217,  0.5015,  0.5098,  0.5021,  0.5345,  0.5090,
         0.5305,  0.4969,  0.4808,  0.4973,  0.5268,  0.4640,  0.5419,
         0.4620,  0.4622,  0.4953,  0.5089,  0.5229,  0.5386,  0.5012,
         0.5334,  0.5065,  0.4312,  0.4776,  0.4992,  0.5183,  0.5045,
         0.4465,  0.4530,  0.4795,  0.5046,  0.5025,  0.5216,  0.5234,
         0.4760,  0.4681,  0.5157,  0.5073,  0.4754,  0.5017,  0.4272,
         0.5002,  0.5356,  0.5509,  0.5239,  0.4713,  0.5309,  0.5194,
         0.4762,  0.4906,  0.5091,  0.5077,  0.485

tensor(0.6348) 34 196
tensor(0.6356) 79 136
tensor(0.6583) 24 118
tensor(0.6418) 10 196
tensor(0.6467) 16 174
tensor(0.6344) 95 182
tensor(0.6529) 4 167
tensor(0.6383) 11 177
tensor(0.6456) 114 144
tensor(0.6388) 56 159
tensor(0.6528) 70 181
tensor(0.6316) 11 146
tensor(0.6246) 11 195
tensor(0.6393) 79 156
tensor(0.6321) 21 201
tensor(0.6390) 12 130
tensor(0.6405) 15 136
tensor(0.6278) 11 145
tensor(0.6518) 15 185
tensor(0.6507) 57 126
Saving players:
tensor([ 0.4784,  0.5691,  0.5105,  0.5049,  0.5063,  0.5207,  0.4829,
         0.5363,  0.5012,  0.4780,  0.5017,  0.4999,  0.4480,  0.5571,
         0.4555,  0.4529,  0.4554,  0.5088,  0.5373,  0.5170,  0.5260,
         0.5550,  0.5038,  0.4637,  0.5219,  0.4932,  0.5046,  0.4599,
         0.4710,  0.4627,  0.4758,  0.5419,  0.4489,  0.5192,  0.5472,
         0.4461,  0.4570,  0.5572,  0.5071,  0.4986,  0.5010,  0.4883,
         0.4930,  0.5247,  0.5716,  0.5410,  0.4960,  0.5373,  0.5037,
         0.4679,  0.4926,  0.4862,  0.5092,  0.

tensor(0.6341) 9 189
tensor(0.6290) 123 144
tensor(0.6305) 118 145
tensor(0.6391) 4 168
tensor(0.6518) 55 162
tensor(0.6426) 22 118
tensor(0.6510) 6 140
tensor(0.6248) 56 149
tensor(0.6125) 7 195
tensor(0.6455) 3 155
tensor(0.6188) 7 114
tensor(0.6209) 26 151
tensor(0.6112) 89 176
tensor(0.6300) 12 139
tensor(0.6551) 6 191
tensor(0.6219) 69 141
tensor(0.6157) 4 160
tensor(0.6254) 10 182
tensor(0.6022) 174 174
tensor(0.6331) 16 154
Saving players:
tensor([ 0.4800,  0.6004,  0.5050,  0.5493,  0.5166,  0.5008,  0.4131,
         0.5619,  0.5010,  0.4504,  0.4342,  0.4976,  0.3758,  0.5968,
         0.4019,  0.4787,  0.3988,  0.5313,  0.5604,  0.4397,  0.4980,
         0.5585,  0.5449,  0.4408,  0.5930,  0.4457,  0.5418,  0.3793,
         0.4594,  0.4387,  0.4522,  0.5303,  0.4155,  0.5655,  0.4996,
         0.4311,  0.4433,  0.6085,  0.5456,  0.4788,  0.4769,  0.5076,
         0.4785,  0.5710,  0.6023,  0.5603,  0.4787,  0.5351,  0.5582,
         0.4629,  0.5136,  0.5225,  0.4586,  0.4864,

tensor(0.6522) 22 161
tensor(0.6319) 44 126
tensor(0.6089) 10 148
tensor(0.6429) 7 190
tensor(0.6282) 8 157
tensor(0.6365) 79 150
tensor(0.6283) 7 168
tensor(0.6282) 18 156
tensor(0.6372) 70 194
tensor(0.6282) 11 120
tensor(0.6531) 24 166
tensor(0.6295) 18 120
tensor(0.6331) 37 178
tensor(0.6228) 11 100
tensor(0.6254) 19 203
tensor(0.6097) 31 144
tensor(0.6044) 79 182
tensor(0.6209) 18 138
tensor(0.6418) 6 114
tensor(0.6241) 69 146
Saving players:
tensor([ 0.4663,  0.6596,  0.4801,  0.5712,  0.5092,  0.5538,  0.4274,
         0.5286,  0.5389,  0.4308,  0.4515,  0.5578,  0.3920,  0.5696,
         0.4279,  0.4082,  0.4133,  0.4526,  0.5846,  0.4655,  0.5256,
         0.5490,  0.5591,  0.4312,  0.6005,  0.4705,  0.5630,  0.4108,
         0.4109,  0.5017,  0.4270,  0.5119,  0.3828,  0.5391,  0.5464,
         0.4517,  0.4401,  0.5992,  0.5257,  0.5151,  0.4080,  0.4829,
         0.4837,  0.5694,  0.6510,  0.5978,  0.5086,  0.5347,  0.4916,
         0.5069,  0.5260,  0.5363,  0.4295,  0.4551

tensor(0.6339) 5 172
tensor(0.6296) 6 153
tensor(0.6062) 39 118
tensor(0.6165) 88 140
tensor(0.6046) 100 218
tensor(0.6103) 18 141
tensor(0.6296) 23 132
tensor(0.6188) 11 221
tensor(0.6189) 22 101
tensor(0.6164) 24 125
tensor(0.6306) 14 153
tensor(0.6220) 29 157
tensor(0.6429) 136 212
tensor(0.6178) 6 131
tensor(0.6070) 0 130
tensor(0.6267) 18 145
tensor(0.6364) 10 158
tensor(0.6135) 56 128
tensor(0.6213) 7 177
tensor(0.6170) 56 138
Saving players:
tensor([ 0.4471,  0.5490,  0.4775,  0.5257,  0.5361,  0.5612,  0.4571,
         0.5945,  0.5299,  0.4278,  0.4174,  0.5519,  0.4123,  0.5697,
         0.4586,  0.4561,  0.4969,  0.5103,  0.5713,  0.5010,  0.4934,
         0.5574,  0.5054,  0.4075,  0.5472,  0.4604,  0.5583,  0.4259,
         0.3840,  0.4398,  0.4572,  0.5285,  0.4822,  0.5878,  0.5268,
         0.4498,  0.3924,  0.5213,  0.4905,  0.5033,  0.4796,  0.4446,
         0.4753,  0.5310,  0.5940,  0.5542,  0.4502,  0.6159,  0.5548,
         0.4441,  0.4959,  0.5093,  0.4564,  0.451

tensor(0.6396) 21 90
tensor(0.6139) 11 157
tensor(0.6322) 16 182
tensor(0.6177) 56 191
tensor(0.6256) 16 163
tensor(0.6094) 11 113
tensor(0.6163) 8 139
tensor(0.6160) 49 229
tensor(0.6165) 4 141
tensor(0.6311) 29 210
tensor(0.6326) 33 263
tensor(0.6073) 56 169
tensor(0.6192) 3 173
tensor(0.6339) 3 153
tensor(0.6138) 4 146
tensor(0.6458) 0 196
tensor(0.6158) 16 139
tensor(0.6084) 56 212
tensor(0.6330) 99 145
tensor(0.6017) 56 183
Saving players:
tensor([ 0.4145,  0.5484,  0.4613,  0.5411,  0.5064,  0.5647,  0.4377,
         0.5729,  0.5332,  0.4379,  0.4348,  0.5407,  0.4184,  0.6003,
         0.4675,  0.4554,  0.5041,  0.4954,  0.5505,  0.5188,  0.5022,
         0.5530,  0.5238,  0.4028,  0.5210,  0.4707,  0.5779,  0.4436,
         0.4010,  0.4319,  0.4364,  0.5231,  0.4947,  0.5771,  0.5096,
         0.4309,  0.4079,  0.5053,  0.4781,  0.4768,  0.4962,  0.4270,
         0.4775,  0.5272,  0.6120,  0.5232,  0.4620,  0.5856,  0.5628,
         0.4413,  0.5155,  0.5020,  0.4805,  0.4749,  

In [ ]:
check_order = list(range(0, player_size**2))

In [ ]:
print(check_order)

In [ ]:
shuffle(check_order)

In [ ]:
check_order